## Exercise 1

In [83]:
# import params

# SCIPER = 374999
# id = 374999
# seed = 13799881773906767546

###### Exercise 1
Q1_param=[-35, 98]
Q1_p=434252269029337012720086440207
Q1_P=[16378704336066569231287640165, 377857010369614774097663166640, 1]
Q1_Q=[410636110197328934180889905758, 132255602069616264622759788969, 1]

## Part 1

In [100]:
# Define the elliptic curve E
K = GF(Q1_p)
E = EllipticCurve(K, Q1_param)

# Define the points on the elliptic curve
Q1_P = E.point(Q1_P)
Q1_Q = E.point(Q1_Q)


In [101]:
def compute_embedding_degree(Q1_p, M):
    k = 1
    while power_mod(Q1_p, k, M) != 1:
        k += 1
    return k

M = E.order()
k = compute_embedding_degree(Q1_p, M)
assert power_mod(Q1_p, k, M) == 1
print(k)

2


## Part 2

In [102]:
# Define the elliptic curve E
K.<y> = GF((Q1_p,2))
E = EllipticCurve(K, Q1_param)

# Define the points on the elliptic curve
Q1_P = E.point(Q1_P)
Q1_Q = E.point(Q1_Q)


Q1b_S1=E.point([16378704336066569231287640165, 377857010369614774097663166640, 1])
Q1b_S2=E.point([6484932502307731637000689302*y + 240043188910717297011740428570, 220475620154759278012544775690*y + 260028317915706903708894824839, 1])

In [103]:
Q1b_S1.weil_pairing(Q1b_S2, Q1b_S1.order())

89184306115119237310377996284*y + 209092368423180334218045696812

In [104]:
R = E.random_point()

while Q1_P.weil_pairing(R, Q1_P.order()) == 1:
    R = E.random_point()
print(R)

a = Q1_Q.weil_pairing(R, Q1_P.order())
b = Q1_P.weil_pairing(R, Q1_P.order())

n = a.log(b)
assert Q1_Q == n * Q1_P

print(f"n = {n}")

(237679314568385138356484160619*y + 289375168244262372281421327186 : 336630570902858696642920323573*y + 14756933649395684808496765050 : 1)
n = 8386275475293703059


# Exercise 2

In [96]:
###### Exercise 2
Q2a_pk=30191450562300985826543871597793035036024846752133137164034820611558441083676
Q2a_p=65198874414494335094231367700284735677717209413817862005917246140593413424021
Q2a_g=37824483197751273988245925781168724650479270951757663957375547064834109552532
Q2a_N=12
Q2a_C=[(54392957776384978324389912892972510646395957526451184776378541083540068342362, 19768655832980648844188936717532790102814386460263628793430086824090783195006), (17353874669593626239798825304618632173512855461581424702136847770533333813166, 43813277399230896758610425022109283421460521095049043070666212054973578565773), (62607150780409280477067286297208392867035867006142667167473429128455430608382, 4974442893020001341934776093230053866223983702094308041404702906599692318528), (12720859200276553597891074524393721023963845537312926750655919707306237166711, 30618118440944313572912175632942319303500624544934556858406294739533448891281), (46945807040363339406954090555404530106023343462352103940542831047878225626382, 23923875936880773761097634811292087190861852442281974331141916373513422722022), (7463595576304333501434815558575162993103966014105155352006560951438128325981, 63475113727853414456076919530025718579429926293283329315687670198305162300240), (19545188257333903108888572705311047305166937546221524766969582966281827654344, 2751954077539300006127635900209866238272707691569941467300104760824263741583), (20184376746354151031883852159244303580416813180422417618242290610668114957988, 9554378979352449979075542242007614746630880899308674539533002326689162523608), (43310734883759567448179107592953106419629714214004585960375061954731041979418, 59657403627018616310621539522837170925239835046747675103791925782287781652885), (54474411663095214919160936112761160341874549384889793076410454551051079012854, 25670864765701438900159300365922126199002816966410698578896327340243380746143), (24025919084588527423040011588864325830212831965161287967505160457995170354783, 64677370332000177596393795692905769910868221487955026392200202565038762810405), (59607911612993404058004079768485768364165355925673041817626548154449420624176, 39380131149205272345804148894565942130590048659550661721449189213332353640526)]

### Test exercise 2 approach with random ratings:

In [97]:
import random

# rand_ratings = [F(random.randrange(Q2a_p)) for i in range(Q2a_N - 2)] + [F(Q2a_p)] + [F(ord_g)]
rand_ratings = [F(1) for i in range(Q2a_N - 2)] + [F(890)] + [F(23890)]


# Manual SUM

actual_rand_sum = F(0)
for r in rand_ratings:
    actual_rand_sum += r
    
print("Print manual sum of ratings")
print(type(actual_rand_sum))
print("raw is ", actual_rand_sum)
print("mod p is ", F(actual_rand_sum))
print()


# Manual PRODUCT

actual_rand_prod = F(1)
for r in rand_ratings:
    actual_rand_prod = r * actual_rand_prod

print("Print manual product of ratings")
print(type(actual_rand_prod))
print("raw is ", actual_rand_prod)
print("mod p is ", F(actual_rand_prod))
print()


# Create Us and Vs

rand_Us = [power_mod(Q2a_g, rat, Q2a_p) for rat in rand_ratings]
rand_Vs = [power_mod(Q2a_pk, rat, Q2a_p) * rat for rat in rand_ratings]

print("Print about randum Us and Vs")
print("type Us ", type(rand_Us[0]))
print("type Vs ", type(rand_Vs[0]))
print()


def solve(Us, Vs, pk, p, g):
    
    F = Zmod(p)
    g = F(g)
    
    prodU = F(1)
    for u in Us:
        prodU = prodU * u
        
    prodV = F(1)
    for v in Vs:
        prodV = prodV * v
        
    sumR = discrete_log(prodU, g, F.order())
    print("discrete log to get sum from exponent, sum is ", sumR) 
    assert g^sumR == prodU
    
    prodU_inverse = power_mod(pk, -sumR, p)
    prodR = prodV * prodU_inverse
    
    return prodR


test_prodR = solve(rand_Us, rand_Vs, Q2a_pk, Q2a_p, Q2a_g)

print("Q2a_rmult = ", test_prodR )

assert test_prodR == actual_rand_prod


Print manual sum of ratings
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_gmp'>
raw is  24790
mod p is  24790

Print manual product of ratings
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_gmp'>
raw is  21262100
mod p is  21262100

Print about randum Us and Vs
type Us  <class 'sage.rings.integer.Integer'>
type Vs  <class 'sage.rings.finite_rings.integer_mod.IntegerMod_gmp'>

discrete log to get sum from exponent, sum is  24790
Q2a_rmult =  21262100


### Actually solve Exercise 2

In [98]:
## Exercise 2

F = Zmod(Q2a_p)
g = F(Q2a_g)
ord_g = g.multiplicative_order()
# print(ord_g)
# print(Q2a_p == ord_g)

# Finding the value \sum_{i=1}^{13} r_i

g_pow_sumr = F(1)
for u in Q2a_C:
    g_pow_sumr = g_pow_sumr * u[0]

sumr = discrete_log(g_pow_sumr, Q2a_g, F.order())
print(sumr) # outputs 116 for Valerio
print(f"g^sumr == g_pow_sumr ?\t{g^sumr == g_pow_sumr}")

prod_v = F(1)
for v in Q2a_C:
    prod_v = prod_v * v[1]

prod_r = prod_v * power_mod(Q2a_pk, -sumr, Q2a_p)

Q2a_rmult = prod_r

print("Q2a_rmult from Valerio's code = \n", Q2a_rmult)

U = [c[0] for c in Q2a_C]
V = [c[1] for c in Q2a_C]

helper_prod_r = solve(U, V, Q2a_pk, Q2a_p, Q2a_g)

print("Q2a_rmult from Annamira's helper func = \n", helper_prod_r)


136
g^sumr == g_pow_sumr ?	True
Q2a_rmult from Valerio's code = 
 10156468998673116498816205673743744315569051428930841746931420107890212344843
discrete log to get sum from exponent, sum is  136
Q2a_rmult from Annamira's helper func = 
 10156468998673116498816205673743744315569051428930841746931420107890212344843


# Exercise 3

In [94]:



###### Exercise 3
Q3a_N=139318878921153532751499562761861928446507405098649008398219509064685868413912856527950494349759160792127809267494353909062242919460654617558230464503523072853970051251389009657389828676782408653856765866847406701730174973965838054507050235650693427895360582038941078988286943128651239704127657219089609607351
Q3a_L=[71428917482626692140169982067862643920420275674669283364745508522809226069141807246524012607121901689983549896702173005394057157599972774611925964105467629064863660890458158323855562269186549518715877956626640975766345665444607083997104539023122776747077177264746821943616443140923900587609056528445625902170, 71428917482626692140169982067862643920420275674669283364745508522809226069141807246524012607121901689983549896702173005394057157599972774611925964105467629064863660890458158323855562269186549518715877956626640975766345665444607083997104539023122776747077177264746821943616443140923900587609056528445625892864, 127928008217276346167336361577656149625995083920413330033640857276488463844161655245001647533047423098521725503859508223718509333968445586126099209962932287246287906410910906759923368271156613625465043826961541638989424258943723093052260395099615387150476380996054707929519308722244956640308340832093776543744]
Q3a_y=19735889032053983154057868876073728216664903734291099482618569146771209013497758865267510952598203047756225193926234102261865551167876941715818029461871888765765060092513537402467007462085036102443934130209613019358047841116721788655675144946688361076515275797917029535600762271496234408860381764181628353362
Q3b_N=785455929752250522510968138141361402301811635023609801152518364528177179650609093778105734218115813442696018896530054015238461280519862421112736780897415576088605371696682805301898911041041149042871203201686638380070296950816804369202647688870766688169371759522803509101865592404353722456402111946669444724350138289519882472154510987057453867921860064773319705847162494416103066831184794711562311087497865088991332741717776495803471404766877350656958123863845239
Q3b_PKS=[1164907164064357708965399500202243790077074149817604819592904535908998512148086958786859711271377913229928370589125687022735503771248272549827584593122453462586861786496337484569590458404071561321105286842186564027185276289630736323333492441086807952976519476361868440872810410447022118004037755990151267468556644982902385596305141304583410007915968617648120943322419928131131114580422072033102418158649773146228572114068086818665150207424574106315370386507350889, 468270446683905721500884480752500023705184737325622702899582882539232347381876220908986390955708831438292780607185724381365469497232336497253465653724170946005206547207366900495608257477867974885263092446027206742236180687438117865474584597891314841111528379354935134803369084430785767768651717479980226320702235149144585993301495922749760111829416320438401281419187841210865379958981951203868739344342128584968670299614096128575887458843042462504664910403097093, 1236666354031710786604529836174929550381473797866064003455294926846988440679992579549389458478403717611944461483526497463395614790204651372912982460800447347287403374380810347232066969058290546594239878629531370442913341882058021847594232729804261069059708194771058431492316614028840468714232864233146486052209981169241123237673672969620079904262081345445390495693217411585461168536120885199461893930428168048505460269202879879167282615767274793967592874253740881, 1042182814749821519341778861179460655273130025330448386417628462760508584513024963301742856940565174577296750580843243197958740321685697090255347890192788769606413307728641493327767624975881035937270022100710937817420681376932949094501474294702376254564918337704056194321543399716559149337645510620173300776273195596010743571134630433761122248830789560807353924860234663265113852387457132401404052931718895031044905898826629759455484764156798125452352980821080239, 1134913015248905205618091569128768220349757454420896925865697698272850703325847631941774521542647717533772528494600448606742960800566545171307153129154660598504710805864540668958037650947123434131967011025389376049916092720130556999956245242017065286447131614769821088680921406711332738075243513783902600326658771595612850802962641451901173044298696969931193180967218000563305759279537948049616971466027097282576887722998365808188760840986068944111890991310345599, 783000050554522670031566815595162921427451617893047255623356841861240688722344792729952979730519169060193732514267013800093318728168425531598594227918937509267298925011322932433183122882019914215973226417193842645892430841892732606971656763632379285676563084866571132096350725901948022387779254838239973689129065452336002213324318767963383306591317052345770916847913033360174248159182313261161049686898002571003270932460471690626210134248760791559844721743706489, 820081629430145140818741376680274918765262628123086606281085309505716302114235496024160350976053797595801443646755668853023656853351469875712020105187431026026964754715196764996542463516384040650338946105901143685949514862778592836614196264903687281016842259655710583585442595165671031293076013660478362016075472389444736782019426629147011139110273748209801447467962754308909501379566832648719161957029023379316878173066537567329111103781915743269511027028378687, 650439752160096119674242494652745619842751298749469294227275073519964278243056438945880357699382691583126482489107175168470439920618680554880651489509514620140308376019375723633417274627277441810940406287109115792056476665283519094477082835691692440821660983404109549580767313822374156485535637686576763079566009625106721808439309981314999768437339655925049309416637242190202080792841392046239465399056732426264470565127050084697343263434910401161331116491826541, 550776555246835460684555818814739742768674533381182517677734017370557083412111480172487583025415970242443001365389857774242915671293446148074461517895312830726653586438861574212056951413461913919107694357531068285372267576770564453365774794417382888196574009821940534040924551154900999572191084352640945102797603971453932130456973450477700283634130971976094955788007645174910500984196154079076292821583777209865845751105966846204890141736599777981288436801432611, 1071901523507501399871608426872735151520775391492470235226576397059700344377909516160000232560406646866445324058134149180508164406655417845199444642638797467599738409623851791356754177959107576213971816450666135321367064310580879718425849775935914305973789817819583377395020525556348336629483257771008591289526156841659144386446116746574697345516545993467540321672455843691822071900696867283891129686089706029597145809291457139598983131184065460249027572765967187, 1378996202972889268133627459678406767997339712952488662543189521170112315319623963027050932093126238232859617038401116502414403186397286125690062282451843042706953333077002851564877750848219703391559453022651064127980122061542020426867175191056312054266959186115053785847739334771499303451549118354465555761587847536567595530312229876642979966403364280674500381993685436315676777822962082184937179612986815397428009996716324364988444838543754751423596520173239179, 1192689354967446477922160822701324252426025608779638275261781279384071677954575355932903503763265298905581732332094788623846462474430608735464399510557578146316199312460405349060339044041731864860072676615236766353462313842756528100060945775394032364384147906114699581493195949377585324937629857470296677246405376185493409937191038971044230155559938702596513058579075270645450509038273596098554109769800368998529070220529216064237433357623736908592032402061052227, 1457149199069860601442802201674604821405420559768175517588947078805138819940201925808131956551729815833732225240888610121676222038004123504620773256699880835014705626584467919650365468424446935481950025457348223185559620821922910831044786858555789093849716629559686237798956742556342266517670357475710875682159518572017267260061955196973375982474576876390944282902252180330459830963955643884381518376916424459415569084982040159614441930343904936936902587030577697, 908270957937958078141337169762735455136219327456193925741610530212650073390132246412779926207423029938515590323985377280756865280254453681824915248133012538708617436739691233313160367096694369247160915259929297937199939871149039112402938467273234125288839838408393460870436802376549709310985471195882803996984025233180880328075016618519842181496135148012267331247960362616229880025688901792651362690623324762151215751662478724771915409783428037176577554699357459, 580852701855681800672298708765652009555139091268590306491570843567678426067793703496325172806464198023062889707906286861579572512682565352665358735267552160447869476810240194888856793517129284293985154046924460039210530495511177274640551972111043590551130168569806116538973605642043385362736870578162662741419471882782244729956514296121843085960178936153677523139771728146890251165346246324611590184162015792931533532821655671865080011753199735886136187724512461, 1428782895480992979534119881146706671170718936198613034318530432883487146980508957188135993463123653298955608281440580983172407764807720269573229071532558985800769776610728733781326777792519848156752956205749785864960126352771129130493265431518531930071693421582290351049285423960034202044521639220490840502441392463770898026644129649071029770917598591959623265643996611826295359665038667664315263440735405557041348211252301672815911427973698447276890812032467083, 1210331127842145581614119956238752956524488880910097428372760540044472655496951403918999641347444951555114212357582974416863480498696686140973773047003829607879464598306603340791028640488868526989372506107111379360046142449476835949810641652765080713689905797108289855553844944273175161008133265125720566168130237827772407945868294342599893464706484807449296883964554686607868571125312978342475429323005655986921625094215038644303924006448313990553693360618948067, 1409369452947711072859634473614002774327083478827630456908008748923098184351804330055487513648417661174748437380349926945288275372084773831597775065087460560220249905313937894838899728739531700839359685373723351090899274782914643888033835558458062652375659695651147021460385891864559990814292394422554523036040233475817949238479661114191977168568329904212020838306272574609580554142582793205628853230520229887997873900359769584127360441077198428469899244005658527, 977742750288958657101774905235241636969088775763469804489699821162682926315451782411143486492482368352626671600101407323182397776888400768557023266633780785441201527628505340298427955320561753500342494322633216393822611860500302316286142692779293338820063193997107665707705869970157000759571690053185124440227681726695705194872563141813128627917214148723814762059975199627182675372691963167580751686674251031597816554874544532002044543243864651855581530896141763, 843335151153970583803945596407554899355900330890663310871833096365274280531287202309733378866159283796602902472153607731490700322234795573902455492982773741823337417065734909576672026935273632750961860668417040981518064147536436532733937573705715311589546563097665982772235641016861861969229371880669644558886830139368496617048599044829635694246130482699719077090908963964670914655684343868312317094077709729417892240866821335321915864577868317831390448336963541, 1255354524081166581308748499611288652999882426204009309080633985966174117644957754560619587831541100921439690227791810820810842531098597270896380805296418444859869296426517938582076271220259148859082130582039099498601779346634471646441924550030474758074475010058228595384121988496110927311627179788913681982387566906698071836021664965310363640257623658428322994729736830705806271173398569326977019335931180012497950047129292036174309034240363833527917799839712327]
Q3b_PAIRS=[(245404097027133115065908257795347461595444945188794610694296446781317456983093716630056838074427936175501356915478164877320715247970733721543933793592748191799070210503499646275757731117369384901503015688342151970409936437803523500578177456429731778874538518736278502592148116218525169183172123841410307905426901870871613148891037042377811322744349976207077739075909616000365655234507182888101101564823811736729199857176713929834578673540187337222418605645832330, 245404097027133115065908257795347461595444945188794610694296446781317456983093716630056838074427936175501356915478164877320715247970733721543933793592748191799070210503499646275757731117369384901503015688342151970409936437803523500578177456429731778874538518736278502592148116218525169183172123841410307905426901870871613148891037042377811322744349976207077739075909616000365655234507182888101101564823811736729199857176713929834578673540187337222418605645832330), (319894969791601601181608999978294797363240658269244677901552105686148575894342600201213824935973903682788129985565503389196336300453997662309387455676798417276192168566388867639871498432206018573088114116394926813186222247376535714335807360054867727868687758162318733791485672959223744180820475268609415159333847991198126051508948667253951947571578849317265498643567912257865421588283400732997191997849649628502139964078894245686964763115487032812697019032650710, 319894969791601601181608999978294797363240658269244677901552105686148575894342600201213824935973903682788129985565503389196336300453997662309387455676798417276192168566388867639871498432206018573088114116394926813186222247376535714335807360054867727868687758162318733791485672959223744180820475268609415159333847991198126051508948667253951947571578849317265498643567912257865421588283400732997191997849649628502139964078894245686964763115487032812697019032650710), (231970240746503110496670147474116396459346721655549822368310571231543349925616690444560692489872690972640509178663203031307939418986771120801001426479715302582154986440106556223954773041758673016008615916571872714931455080447903460590399312779676736622794767721412006433704998419860226634691660300844571502898265705633151881742351800953128722931277291158719096268731250959105480916091409444820878305427326953821997796115319755491932843820349573190044755647821314, 231970240746503110496670147474116396459346721655549822368310571231543349925616690444560692489872690972640509178663203031307939418986771120801001426479715302582154986440106556223954773041758673016008615916571872714931455080447903460590399312779676736622794767721412006433704998419860226634691660300844571502898265705633151881742351800953128722931277291158719096268731250959105480916091409444820878305427326953821997796115319755491932843820349573190044755647821314), (49978570513461287606862768267783213766616767144016506997791948210670631817877010029862574137021628522287118158047992765090458002753450387981419269985785805643725542633157921448161128517401245115909286655578975859176436554423707471140595450931266203975913712687336891225002757410216050446991085189349717812450239253730740153116336225972702673309448407452151074109988527686636216762999101889386482446316668018767671099981120232827283009574371103682883258536632037, 49978570513461287606862768267783213766616767144016506997791948210670631817877010029862574137021628522287118158047992765090458002753450387981419269985785805643725542633157921448161128517401245115909286655578975859176436554423707471140595450931266203975913712687336891225002757410216050446991085189349717812450239253730740153116336225972702673309448407452151074109988527686636216762999101889386482446316668018767671099981120232827283009574371103682883258536632037), (16970097333966910873347505867556933510150341408177308357266849273015376400878013227410962471791625561972287457197424030142654139277908654567191624792509369126089437616735478635953714065355806672242423911521258203878325968549507906762167053905186064217712575527800679536203420764384173784215583449988768404176305082425296472147175537841812165615749578291019623943263040857489166627177133637816679552661111810132702505519536147309554147511579879786900632775808752, 16970097333966910873347505867556933510150341408177308357266849273015376400878013227410962471791625561972287457197424030142654139277908654567191624792509369126089437616735478635953714065355806672242423911521258203878325968549507906762167053905186064217712575527800679536203420764384173784215583449988768404176305082425296472147175537841812165615749578291019623943263040857489166627177133637816679552661111810132702505519536147309554147511579879786900632775808752), (394969176780239706943982645562162775719385007109823868616321526514214735874203489940959002501667939657414975597285647354336292919024539069598861238437266291335065952425763714347320694877247648356950100661367632094822209731443270088677241290967758114600632871831697783850837752962347049598332553159641131313603961680410037338265634679449755451916207064005091130088071207429237261508678404400830582467922088518475951682834965014860021201380620055927521532737696773, 394969176780239706943982645562162775719385007109823868616321526514214735874203489940959002501667939657414975597285647354336292919024539069598861238437266291335065952425763714347320694877247648356950100661367632094822209731443270088677241290967758114600632871831697783850837752962347049598332553159641131313603961680410037338265634679449755451916207064005091130088071207429237261508678404400830582467922088518475951682834965014860021201380620055927521532737696773), (578012703386301611867861279443215761000013242922761375684538302999769782795335691057117340373106118282793408275418352866310532686914592426403976028138772979521836271372307739796931507432245439031918479416211177005549867839584665611643550975440160707501483910175989757914503779274488434576554983824846808792669813008255938374812820874870364925846155945133171875115426423790516320222725587157885509474396530235029669658701906247608400297663884637882031979263820364, 578012703386301611867861279443215761000013242922761375684538302999769782795335691057117340373106118282793408275418352866310532686914592426403976028138772979521836271372307739796931507432245439031918479416211177005549867839584665611643550975440160707501483910175989757914503779274488434576554983824846808792669813008255938374812820874870364925846155945133171875115426423790516320222725587157885509474396530235029669658701906247608400297663884637882031979263820364), (532031070907351895313766646689577303809849157690987686318456022193655355980228128964469186269142098337217899201596911614887090515061098955037142855860544262661794943835309401063575685222203465727082629494560513223553128170723589081754871068650546505211351785611078858923391943398191160286119821153462022248600628922835547195934084739346805912216689811771960918296219704602394310570757175086905459345315619540473082134454715238383051845504213975497913567676136843, 532031070907351895313766646689577303809849157690987686318456022193655355980228128964469186269142098337217899201596911614887090515061098955037142855860544262661794943835309401063575685222203465727082629494560513223553128170723589081754871068650546505211351785611078858923391943398191160286119821153462022248600628922835547195934084739346805912216689811771960918296219704602394310570757175086905459345315619540473082134454715238383051845504213975497913567676136843), (518811759053934186241165403100628502596655786834308094661995049251660753032688925990568252210910185690826349236324331376006787128466680684998482663037025301850698290181138256322933401834882431998983855529024042771009076518128900541291316910856140696517454075469235074096303982498422144574188389554512204284628009175416760585211309273576435802383774643517060869865313950933327671982721294030935550604548225239861121135445546195909559666634497911573040406670697369, 518811759053934186241165403100628502596655786834308094661995049251660753032688925990568252210910185690826349236324331376006787128466680684998482663037025301850698290181138256322933401834882431998983855529024042771009076518128900541291316910856140696517454075469235074096303982498422144574188389554512204284628009175416760585211309273576435802383774643517060869865313950933327671982721294030935550604548225239861121135445546195909559666634497911573040406670697369), (85106609603998340462451460301183296202629350504229868378183987546831146980703730496828247275799791262853480575023058312130699199286785573713117469667678501163352932232585469235709557238516102420576602974573708016482212430645236734284130433812421104949545078943902285810988116811551328099367398656631721356109816520909256073500795447306356750628740205538369914349466077209614319938778337588029423929241630996132253832969225887411879767306983454244635774694355144, 85106609603998340462451460301183296202629350504229868378183987546831146980703730496828247275799791262853480575023058312130699199286785573713117469667678501163352932232585469235709557238516102420576602974573708016482212430645236734284130433812421104949545078943902285810988116811551328099367398656631721356109816520909256073500795447306356750628740205538369914349466077209614319938778337588029423929241630996132253832969225887411879767306983454244635774694355144), (654927939289915126870039377015968033618503972363342644017512614097238414939783931379014481398517987413696984434300492290120122527380409075009335441392031834471129040081153416152991089328855770317917632397167287189672053654902376384118460541524179831831281755484098622012308799550182833872440482314140383571209987259446530147991381047352315959887460576700822063749774699933650187319888740968704932284299594419724470047738465528971764671547677429705828993663833269, 654927939289915126870039377015968033618503972363342644017512614097238414939783931379014481398517987413696984434300492290120122527380409075009335441392031834471129040081153416152991089328855770317917632397167287189672053654902376384118460541524179831831281755484098622012308799550182833872440482314140383571209987259446530147991381047352315959887460576700822063749774699933650187319888740968704932284299594419724470047738465528971764671547677429705828993663833269), (665129549617069093019834635738722703393135689772317377659299454216549368133577773633616923288719503522676671277988107139913414162311371442100682952010771640645019713288551886597699584208421081457351578964265344851199989325856645982773710839964269438583941741466990138093331332069189836277901735121807671230295034525052878645430390373128806020238261181504273700682947122033718904337274302637569501365984762979769697028373817524989621896454897774416900608310571916, 665129549617069093019834635738722703393135689772317377659299454216549368133577773633616923288719503522676671277988107139913414162311371442100682952010771640645019713288551886597699584208421081457351578964265344851199989325856645982773710839964269438583941741466990138093331332069189836277901735121807671230295034525052878645430390373128806020238261181504273700682947122033718904337274302637569501365984762979769697028373817524989621896454897774416900608310571916), (528956129652432624842206020518608010257633572826035790504068157689288639229441094323090140346166668334766440568707371590117468171701133384008476626042978167785906012422609828984768956547792627343601466273917382596871672315759035723392497942824326139164575651930331673488924896592369695364202578858914044454119810957264499671093032443033702554874623140823772516925386006798983217557495963398741102590433209510170953516055885960891970624150799007235271572920274843, 528956129652432624842206020518608010257633572826035790504068157689288639229441094323090140346166668334766440568707371590117468171701133384008476626042978167785906012422609828984768956547792627343601466273917382596871672315759035723392497942824326139164575651930331673488924896592369695364202578858914044454119810957264499671093032443033702554874623140823772516925386006798983217557495963398741102590433209510170953516055885960891970624150799007235271572920274843), (81445954678629602153980143299994973098162064582599490042199537534335890809314446487454363923844428253239384475740357059527314667373971549729471757859965997447152516103274293742772848812841517819227043319017774294516906088553062616217288165258007351040798977730110846151950250750624411729096973045752704008704396478097228915059259957335188650869407338632770100427049122680373941751199320781151872528424809447245782793500655096916802990751799368785272771410772712, 81445954678629602153980143299994973098162064582599490042199537534335890809314446487454363923844428253239384475740357059527314667373971549729471757859965997447152516103274293742772848812841517819227043319017774294516906088553062616217288165258007351040798977730110846151950250750624411729096973045752704008704396478097228915059259957335188650869407338632770100427049122680373941751199320781151872528424809447245782793500655096916802990751799368785272771410772712), (217464463812504571992857930330910112233561678756406858497276872489829784579532745564570411796113817737610431089773634012347653146279088126219290705298623598712613609712456212334784367819424637621846935550591358809322813906672520009452287894602808287562906304526176533921377669107564390550498312371837583437461523290783107883266976247695583698229005221879245524977401203645432120042131249954417846863944802717676042446187992721728840382915016622253307409654049598, 217464463812504571992857930330910112233561678756406858497276872489829784579532745564570411796113817737610431089773634012347653146279088126219290705298623598712613609712456212334784367819424637621846935550591358809322813906672520009452287894602808287562906304526176533921377669107564390550498312371837583437461523290783107883266976247695583698229005221879245524977401203645432120042131249954417846863944802717676042446187992721728840382915016622253307409654049598), (536371497610044467394897540509847136132137847614233320123386286016196683394163792265117941114519566364683159149944115299497763964072504974711366387692117586141036047172062994254625999871071362348294420686744232199528058413304988916355384404255580748332725191334924118535212246248130892923571816676531002760601081603282669373490922289140472193595612908153607729432014841906172720822235901397502065594469880197025582214017631888094203742918431899900107529999755963, 536371497610044467394897540509847136132137847614233320123386286016196683394163792265117941114519566364683159149944115299497763964072504974711366387692117586141036047172062994254625999871071362348294420686744232199528058413304988916355384404255580748332725191334924118535212246248130892923571816676531002760601081603282669373490922289140472193595612908153607729432014841906172720822235901397502065594469880197025582214017631888094203742918431899900107529999755963), (381318674511147597432594950738902794154983669713955150868693078955902947405416342836167025789333360293870092165587274725855136588145947131458345336082044375812387168180001960255279814648085469119317684552210225384296243315957014906996355447345715160492558259386295224348791966882560146245270520530872717180216377724448502793830371659875665164967490844508954897008001293539789407883567642152565680478399262713325632894227481427270223630886360283158759768464464074, 381318674511147597432594950738902794154983669713955150868693078955902947405416342836167025789333360293870092165587274725855136588145947131458345336082044375812387168180001960255279814648085469119317684552210225384296243315957014906996355447345715160492558259386295224348791966882560146245270520530872717180216377724448502793830371659875665164967490844508954897008001293539789407883567642152565680478399262713325632894227481427270223630886360283158759768464464074), (730723570999161133513453126491564705645058848482187460018920922070906886908419740648166897770565491013266748707185180821760555989290145012867179218697375866996292074087356299035956287706689542925937655584174632175803732692030897579162129738592573368499958851057637450002160159335806689244840936260898086566070214172958232194269573395195056188170667209453443995403820055259026656131246884686661102498916075151167793513036767616319280878321191335045579775451484061, 730723570999161133513453126491564705645058848482187460018920922070906886908419740648166897770565491013266748707185180821760555989290145012867179218697375866996292074087356299035956287706689542925937655584174632175803732692030897579162129738592573368499958851057637450002160159335806689244840936260898086566070214172958232194269573395195056188170667209453443995403820055259026656131246884686661102498916075151167793513036767616319280878321191335045579775451484061), (535779688792207757995657861792495566049706870998851032006458432647862577504879476687149895074401696176742863316616076568247046900113598020380650730045226830732951279965365581764606533425091667127793396754958035443919043398776986794260160018331230225263080095445430441129522904207494757948857661482531160572582869271552181242517281429028732197829381755185156727474017402251185252719355058998837306599628543589463673520176738487648395000840577437918281615231904777, 535779688792207757995657861792495566049706870998851032006458432647862577504879476687149895074401696176742863316616076568247046900113598020380650730045226830732951279965365581764606533425091667127793396754958035443919043398776986794260160018331230225263080095445430441129522904207494757948857661482531160572582869271552181242517281429028732197829381755185156727474017402251185252719355058998837306599628543589463673520176738487648395000840577437918281615231904777), (253833861906991143406684843516915733723639052823468963953440277163492749983104515258413208085660034149145641506212205404151086108689134823660186063099231226538262824110875661242620501852609503755803566413824072870947842009754302209193647627982609302092938946320387430641868471288720081292169108537387954195647488604022299513183417322716178787615459228525583120681803648718947220064577232806146259515081619239440261960968916878597703810120048848644746251830880421, 253833861906991143406684843516915733723639052823468963953440277163492749983104515258413208085660034149145641506212205404151086108689134823660186063099231226538262824110875661242620501852609503755803566413824072870947842009754302209193647627982609302092938946320387430641868471288720081292169108537387954195647488604022299513183417322716178787615459228525583120681803648718947220064577232806146259515081619239440261960968916878597703810120048848644746251830880421), (569013491221875851794112668019998874358443906796692188901402608758349522004347675587892923741608269798151384373088834023002293454890197202260257714680692185083012477295505850426509033114211825708749101312337898548056411451579808911030155347392359566346423741540124835824052127457596578724696567604875246409533898414896580607874212195020807235022595950390462404351681933870275133143900298895812637010693528447096930901653653189681183036169766928401016131249016533, 771227673112661074791543410882450238284939597456788469281260151109161007285647042732071221190588487761167771217360913157986049844451966783011090147703520518516040331512247054023352450562981273638459913806587496025986240332364202215536293416966830497816160479773128291898568071828291981916543821116552023335000480282597040890846554815207541645486885745017420402628733319970790349219242101805361105211348926850823598297089849912405285527327039218619817797783810876)]
Q3b_CT_RSA=16452436860146144733196215417011814257411501286742408431780682439602527968419290721241167980897310414213794907798625
Q3b_CT_AES=b'\x8cW\xa4ZL2\xfd\xe6\xf5\xb3\xd2\x02\xa3{H\xd6\x1258\xef\xfd\xc0/\xdd\x81\xf1z\xbb\x1c(.0/\xffd\xa0C\xdd\x1d\xa9'
Q3b_CT_NONCE=b'\xea\x96\x9a\xe7P\xb0(\xadf\xecD\x9d\x16\x99`\x10'
Q3b_CT_TAG=b'\x8e\xf8r\x8dbB\xda\xca\xb4\x18\x83\x8aZ\xfb<7'



## Part 1

# get last element of the list of points

last_elem_bits = Q3a_L[-1].bits()

# brute force last 15 bits until we find a nontrivial root of 1

bitlen = len(last_elem_bits)
nontrivial_root = 0

for n in range(0, 2**15):
    
    bits = Integer(n).bits()
    
    L = len(bits)
    padded_bits = bits + ([0] * (15 - L))
    
    attempt = padded_bits + last_elem_bits[15:] 
    assert len(attempt) == bitlen
    int_val = ZZ(attempt, base=2)
    square = power_mod(int_val, 2, Q3a_N)
    if square == 1:
        nontrivial_root = int_val
        break
        

    
print("Nontrivial root of 1 is ", nontrivial_root)
print("Root squared mod N is ", power_mod(nontrivial_root, 2, Q3a_N))

# find GCD of N and the nontrivial root minus 1

nontrivial_factor = GCD(Q3a_N, nontrivial_root - 1)
P = nontrivial_factor
Q = ZZ(Q3a_N / P)

print("First nontrivial factor of N is ", P)
print("Is P prime?  ", P.is_prime())
print("Second nontrivial factor of N is ", Q)
print("Is Q prime?  ", Q.is_prime())

print("Do the two nontrivial factors found multiply to N?  ", P*Q == Q3a_N)

print()
print("Finding QRs mod P and Q...")

Zn = Zmod(Q3a_N)
Zp = Zmod(P)
Zq = Zmod(Q)

QR_modP = power_mod(Q3a_y, (P+1)/4, P) # square_root_mod_prime(Q3a_y, p=P)
other = sqrt(Zp(Q3a_y))

assert power_mod(QR_modP, 2, P) == (Q3a_y % P)
assert other == QR_modP

QR_modQ = sqrt(Zq(Q3a_y))

assert power_mod(QR_modP, 2, P) == (Q3a_y % P)
assert power_mod(QR_modQ, 2, Q) == (Q3a_y % Q)

print("QR of y mod P is ", QR_modP)
print("QR of y mod Q is ", QR_modQ)

print()
print("Combining QRs of Y with CRT...")
print()

root00 = crt(ZZ(QR_modP), ZZ(QR_modQ), P, Q)
root01 = crt(ZZ(QR_modP), ZZ(Zq(Q - QR_modQ)), P, Q)
root10 = crt(ZZ(Zp(P - QR_modP)), ZZ(Zq(QR_modQ)), P, Q)
root11 = crt(ZZ(Zp(P - QR_modP)), ZZ(Zq(Q - QR_modQ)), P, Q)

rootsY = sorted([root00, root01, root10, root11])

# make sure no roots are duplicates
assert(len(set(rootsY)) == 4)

assert power_mod(root01, 2, Q3a_N) == Zn(Q3a_y)
assert power_mod(root10, 2, Q3a_N) == Zn(Q3a_y)
assert power_mod(root10, 2, Q3a_N) == Zn(Q3a_y)
assert power_mod(root11, 2, Q3a_N) == Zn(Q3a_y)

# for i in range(4):
#     print(f"Root number {i} of Y is:")
#     print(rootsY[i])
#     print()
    
print("Roots in ascending order are: ", rootsY)



Nontrivial root of 1 is  127928008217276346167336361577656149625995083920413330033640857276488463844161655245001647533047423098521725503859508223718509333968445586126099209962932287246287906410910906759923368271156613625465043826961541638989424258943723093052260395099615387150476380996054707929519308722244956640308340832093776545405
Root squared mod N is  1
First nontrivial factor of N is  13169420695771863846677571050430875414415295065454283167144895221620406447131662042099750516480336983695453551203674736697999446989548142691858914162956767
Is P prime?   True
Second nontrivial factor of N is  10578967909035121752358381405508655860431656165486745446246696102991019817035008002078381129073245564467182462917312046774196639554629163104731621051866153
Is Q prime?   True
Do the two nontrivial factors found multiply to N?   True

Finding QRs mod P and Q...
QR of y mod P is  10334601205862665267786231347673029792298708613618476987678556195204055538404492330268586624884395330211247178144657029

In [95]:
## Part 2

p = 0
for i in range(len(Q3b_PKS)):
    if gcd(Q3b_PKS[i], Q3b_N) != 1:
        p = gcd(Q3b_PKS[i], Q3b_N)    # since you don't break the loop, we take the last p found

print("P found is ", p)
print("P found is prime? ", p.is_prime())
qr = Q3b_N / p
print("QR is ", qr)
print("QR found is prime (it shouldn't be)?", qr.is_prime())

# This tells us that Q3b_N = p * q * r

glitch_X, glitch_Y = 0, 0

# finds pairs of 
for i, j in Q3b_PAIRS:
    if i != j:
        glitch_X = i 
        glitch_Y = j 

print(glitch_X, glitch_Y)

from sage.arith.misc import XGCD
Z_N = Zmod(Q3b_N)

Z_QR = Zmod(qr)
Z_P = Zmod(p)

random_mod_p = Z_P(glitch_Y)
X_mod_p = Z_P(glitch_X)
print("Did the glitch occur while decrypting mod p ?\t", random_mod_p != X_mod_p)

random_mod_qr = Z_QR(glitch_Y)
X_mod_qr = Z_QR(glitch_X)
print("Did the glitch occur while decrypting mod qr ?\t", random_mod_qr != X_mod_qr)

# Computing the second factor of 
r = GCD(Q3b_N / p, glitch_Y - glitch_X)
print("r = ",r)
q = Q3b_N / r / p
print("q = ", q)
print('Is r prime ?\t', Integer(r).is_prime())
q = Q3b_N / r / p
print('Is q prime ?\t', Integer(q).is_prime())

print(f"r * q * p == Q3b_N ?\t{r * q * p == Q3b_N}")

# Computing the private key d

from sage.arith.misc import XGCD

Z_phi = Zmod((p - 1)*(q - 1)*(r - 1))
d = Z_phi(XGCD(3, (p - 1)*(q - 1)*(r - 1))[1])

# Checking that the computed private key d is valid
print(f"Is d a valid private key?\t{Z_phi(d*3) == 1}")

rec_k_aes = power_mod(Q3b_CT_RSA, d, Q3b_N)
# print(Z_N(power_mod(Q3b_CT_RSA, d, Q3b_N)))
print("rec_k_aes =", rec_k_aes)

P found is  9136941394589939168863778988883690151544243332265914435108778243115204302081615090671280598206353090934033624779011715778943416508624522200218282869982009
P found is prime?  True
QR is  85964864589951914245941873541152660081906770496107564365864514694554431315453543291450351413037938241874677897644838895000744632859890188183743966173095757466355836292860851365092315830998989988195103865935415002045040654352530063455281756702351484198912989481151265132929227732511311690458226702911415591471
QR found is prime (it shouldn't be)? False
569013491221875851794112668019998874358443906796692188901402608758349522004347675587892923741608269798151384373088834023002293454890197202260257714680692185083012477295505850426509033114211825708749101312337898548056411451579808911030155347392359566346423741540124835824052127457596578724696567604875246409533898414896580607874212195020807235022595950390462404351681933870275133143900298895812637010693528447096930901653653189681183036169766928401016